In [1]:
from parsetrees.expr_parser import webwork_parser
from parsetrees.expr_parser import Eval_parsed
reload(webwork_parser)
reload(Eval_parsed)
import pickle
from collections import deque
from collections import defaultdict
import cluster_functions
reload(cluster_functions)
from itertools import combinations
from datetime import datetime as dt

lex: tokens   = ('Q', 'PHI', 'CHOOSE', 'SQRT', 'PERMUTE', 'VARIABLE', 'NUMBER', 'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'LPAREN', 'RPAREN', 'FACTORIAL', 'LSET', 'RSET', 'COMMA', 'EXP', 'LBRACKET', 'RBRACKET', 'COMPUTE')
lex: literals = ''
lex: states   = {'INITIAL': 'inclusive'}
lex: Adding rule t_NUMBER -> '\d*\.?\d+(E(\+|\-)?\d+)?' (state 'INITIAL')
lex: Adding rule t_newline -> '\n+' (state 'INITIAL')
lex: Adding rule t_VARIABLE -> '(?<![A-Za-z])[ABD-OR-Za-z](?![A-Za-z])' (state 'INITIAL')
lex: Adding rule t_EXP -> '\^|(\*\*)' (state 'INITIAL')
lex: Adding rule t_COMPUTE -> 'Compute' (state 'INITIAL')
lex: Adding rule t_SQRT -> 'sqrt' (state 'INITIAL')
lex: Adding rule t_PHI -> 'Phi' (state 'INITIAL')
lex: Adding rule t_RSET -> '\}' (state 'INITIAL')
lex: Adding rule t_RBRACKET -> '\]' (state 'INITIAL')
lex: Adding rule t_LBRACKET -> '\[' (state 'INITIAL')
lex: Adding rule t_PLUS -> '\+' (state 'INITIAL')
lex: Adding rule t_COMMA -> '\,' (state 'INITIAL')
lex: Adding rule t_LPAREN -> '\

In [3]:
#### Example of flatten ####
parse_tree = webwork_parser.parse_webwork('C(3,5)*9+8')
parse_tree

([['+', [0, 9]],
  [['*', [0, 7]],
   [['C', [0, 5]], [['X', [2, 2]], 3], [['X', [4, 4]], 5]],
   [['X', [7, 7]], 9]],
  [['X', [9, 9]], 8]],
 {})

In [4]:
etree = Eval_parsed.eval_parsed(parse_tree[0])
etree

get_number got (3.0,)
get_number got ['X', 3.0, [2, 2], 'R.0.0.0']
get_number got (5.0,)
get_number got ['X', 5.0, [4, 4], 'R.0.0.1']
Eval_parsed Exception: reduce() of empty sequence with no initial value
Traceback (most recent call last):
  File "parsetrees/expr_parser/Eval_parsed.py", line 140, in eval_parsed
    ans= ncr(int(v1), int(v2))
  File "parsetrees/expr_parser/Eval_parsed.py", line 35, in ncr
    numer = reduce(op.mul, xrange(n, n-r, -1))
TypeError: reduce() of empty sequence with no initial value
get_number got None
Eval_parsed Exception: object of type 'NoneType' has no len()
Traceback (most recent call last):
  File "parsetrees/expr_parser/Eval_parsed.py", line 129, in eval_parsed
    v1=get_number(ev1)
  File "parsetrees/expr_parser/Eval_parsed.py", line 75, in get_number
    if len(ev)==4 and ev[0]=='X':
TypeError: object of type 'NoneType' has no len()
get_number got None
Eval_parsed Exception: object of type 'NoneType' has no len()
Traceback (most recent call last):

In [64]:
#### Example of find_Hits
parse_tree = webwork_parser.parse_webwork('3*5+7')
etree_a = Eval_parsed.eval_parsed(parse_tree[0])
parse_tree_b = webwork_parser.parse_webwork('C(3,1)C(2,1)+7')
etree_b = Eval_parsed.eval_parsed(parse_tree_b[0])
p = {'attempt':'C(3,1)C(2,1)+7', 'answer':'3*5+7', 'att_tree':etree_b, 'ans_tree': etree_a}
print cluster_functions.find_matches(p)
print [int(i)+1 for i in cluster_functions.find_matches(p)[0][0].split('.')[1:]]
print parse_tree_b[0][1][1]
print tree_to_s_exp(parse_tree_b[0][1][1])

get_number got (3.0,)
get_number got ['X', 3.0, [0, 0], 'R.0.0']
get_number got (5.0,)
get_number got ['X', 5.0, [2, 2], 'R.0.1']
get_number got [['*', 15.0, [0, 2], 'R.0'], ['X', 3.0, [0, 0], 'R.0.0'], ['X', 5.0, [2, 2], 'R.0.1']]
get_number got (7.0,)
get_number got ['X', 7.0, [4, 4], 'R.1']
get_number got (3.0,)
get_number got ['X', 3.0, [2, 2], 'R.0.0.0']
get_number got (1.0,)
get_number got ['X', 1.0, [4, 4], 'R.0.0.1']
get_number got [['C', 3, [0, 5], 'R.0.0'], ['X', 3.0, [2, 2], 'R.0.0.0'], ['X', 1.0, [4, 4], 'R.0.0.1']]
get_number got (2.0,)
get_number got ['X', 2.0, [8, 8], 'R.0.1.0']
get_number got (1.0,)
get_number got ['X', 1.0, [10, 10], 'R.0.1.1']
get_number got [['C', 2, [6, 11], 'R.0.1'], ['X', 2.0, [8, 8], 'R.0.1.0'], ['X', 1.0, [10, 10], 'R.0.1.1']]
get_number got [['*', 6.0, [0, 11], 'R.0'], [['C', 3, [0, 5], 'R.0.0'], ['X', 3.0, [2, 2], 'R.0.0.0'], ['X', 1.0, [4, 4], 'R.0.0.1']], [['C', 2, [6, 11], 'R.0.1'], ['X', 2.0, [8, 8], 'R.0.1.0'], ['X', 1.0, [10, 10], 'R.0.1

In [84]:
reload(cluster_functions)
def evaluate(ans, att):
    p = cluster_functions.make_params(ans, att)
    final_pairs = cluster_functions.find_matches()
    if len(final_pairs) == 1 and final_pairs[0][0] == 'R':
        return True
    return False
print evaluate('26^3-25^3', '24^3')
#cluster_functions.show_matching_group_w_variables('3*2+n', 'C(3,1)C(2,1)+n', {'n':[1,2,3,4,5]})

get_number got (26.0,)
get_number got ['X', 26.0, [0, 1], 'R.0.0']
get_number got (3.0,)
get_number got ['X', 3.0, [3, 3], 'R.0.1']
get_number got [['^', 17576.0, [0, 3], 'R.0'], ['X', 26.0, [0, 1], 'R.0.0'], ['X', 3.0, [3, 3], 'R.0.1']]
get_number got (25.0,)
get_number got ['X', 25.0, [5, 6], 'R.1.0']
get_number got (3.0,)
get_number got ['X', 3.0, [8, 8], 'R.1.1']
get_number got [['^', 15625.0, [5, 8], 'R.1'], ['X', 25.0, [5, 6], 'R.1.0'], ['X', 3.0, [8, 8], 'R.1.1']]
get_number got (24.0,)
get_number got ['X', 24.0, [0, 1], 'R.0']
get_number got (3.0,)
get_number got ['X', 3.0, [3, 3], 'R.1']


KeyError: 'attempt'

In [49]:
attempt=p['attempt']
attempt_tree=p['att_tree']
attempt_list=cluster_functions.flatten(attempt_tree,'t')
attempt_list

get_number got (3.0,)
get_number got ['X', 3.0, [0, 0], 'R.0.0']
get_number got (2.0,)
get_number got ['X', 2.0, [2, 2], 'R.0.1']
get_number got [['*', 6.0, [0, 2], 'R.0'], ['X', 3.0, [0, 0], 'R.0.0'], ['X', 2.0, [2, 2], 'R.0.1']]
get_number got (4.0,)
get_number got ['X', 4.0, [4, 4], 'R.1']
get_number got (3.0,)
get_number got ['X', 3.0, [2, 2], 'R.0.0']
get_number got (1.0,)
get_number got ['X', 1.0, [4, 4], 'R.0.1']
get_number got [['C', 3, [0, 5], 'R.0'], ['X', 3.0, [2, 2], 'R.0.0'], ['X', 1.0, [4, 4], 'R.0.1']]
get_number got (2.0,)
get_number got ['X', 2.0, [8, 8], 'R.1.0']
get_number got (1.0,)
get_number got ['X', 1.0, [10, 10], 'R.1.1']
get_number got [['C', 2, [6, 11], 'R.1'], ['X', 2.0, [8, 8], 'R.1.0'], ['X', 1.0, [10, 10], 'R.1.1']]


[[1.0, 't', ['X', 1.0, [4, 4], 'R.0.1']],
 [1.0, 't', ['X', 1.0, [10, 10], 'R.1.1']],
 [2,
  't',
  [['C', 2, [6, 11], 'R.1'],
   ['X', 2.0, [8, 8], 'R.1.0'],
   ['X', 1.0, [10, 10], 'R.1.1']]],
 [2.0, 't', ['X', 2.0, [8, 8], 'R.1.0']],
 [3,
  't',
  [['C', 3, [0, 5], 'R.0'],
   ['X', 3.0, [2, 2], 'R.0.0'],
   ['X', 1.0, [4, 4], 'R.0.1']]],
 [3.0, 't', ['X', 3.0, [2, 2], 'R.0.0']],
 [6.0,
  't',
  [['*', 6.0, [0, 11], 'R'],
   [['C', 3, [0, 5], 'R.0'],
    ['X', 3.0, [2, 2], 'R.0.0'],
    ['X', 1.0, [4, 4], 'R.0.1']],
   [['C', 2, [6, 11], 'R.1'],
    ['X', 2.0, [8, 8], 'R.1.0'],
    ['X', 1.0, [10, 10], 'R.1.1']]]]]

In [4]:
answer=p['answer']
answer_tree=p['ans_tree']
answer_list=cluster_functions.flatten(answer_tree,'c')

combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])
combined_list

[[2.0, 'c', ['X', 2.0, [2, 2], 'R.1']],
 [2.0, 't', ['X', 2.0, [2, 2], 'R.1']],
 [3.0, 'c', ['X', 3.0, [0, 0], 'R.0']],
 [3.0, 't', ['X', 3.0, [0, 0], 'R.0']],
 [6.0,
  'c',
  [['*', 6.0, [0, 2], 'R'],
   ['X', 3.0, [0, 0], 'R.0'],
   ['X', 2.0, [2, 2], 'R.1']]],
 [6.0,
  't',
  [['*', 6.0, [0, 2], 'R'],
   ['X', 3.0, [0, 0], 'R.0'],
   ['X', 2.0, [2, 2], 'R.1']]]]

In [5]:
Hits = cluster_functions.find_Hits(combined_list)
Hits

[([2.0, 'c', ['X', 2.0, [2, 2], 'R.1']],
  [2.0, 't', ['X', 2.0, [2, 2], 'R.1']]),
 ([3.0, 'c', ['X', 3.0, [0, 0], 'R.0']],
  [3.0, 't', ['X', 3.0, [0, 0], 'R.0']]),
 ([6.0,
   'c',
   [['*', 6.0, [0, 2], 'R'],
    ['X', 3.0, [0, 0], 'R.0'],
    ['X', 2.0, [2, 2], 'R.1']]],
  [6.0,
   't',
   [['*', 6.0, [0, 2], 'R'],
    ['X', 3.0, [0, 0], 'R.0'],
    ['X', 2.0, [2, 2], 'R.1']]])]

In [6]:
#### Example of find_dominating_hits ####
cluster_functions.find_dominating_hits(Hits,answer,attempt)

[('R', 6.0, '3*2', 'C(3')]

In [112]:
### Count the number attempts and answer for each part ###
problem_count = defaultdict(int)
problem_part_answer = defaultdict(str)
for d in data:
    if d['answer']:
        problem_count[(d['problem_id'], d['part_id'])] += 1
        problem_part_answer[(d['problem_id'], d['part_id'])] = d['answer']
print len(data)
problem_part_answer

17828


defaultdict(str,
            {(u'1', u'1'): u'B',
             (u'2', u'1'): u'0.05/[sqrt(0.16/200)]',
             (u'2', u'2'): u'Q(1.76777)',
             (u'2',
              u'3'): u'<pre style="text-align:left; padding-left:.2em">yes</pre>',
             (u'2',
              u'4'): u'<pre style="text-align:left; padding-left:.2em">no</pre>',
             (u'5', u'1'): u'-0.02-2/[sqrt(25)]*2',
             (u'5', u'2'): u'-0.03+2/[sqrt(25)]*2',
             (u'5', u'3'): u'-1',
             (u'5', u'4'): u'1',
             (u'5', u'5'): u'1',
             (u'6', u'1'): u'65',
             (u'6',
              u'2'): u'65*(0.5*[(30/65-0.5)/0.5]^2+0.25*[(20/65-0.25)/0.25]^2+0.25*[(15/65-0.25)/0.25]^2)',
             (u'6',
              u'3'): u'65*(1/3*[(30/65-1/3)/(1/3)]^2+1/3*[(20/65-1/3)/(1/3)]^2+1/3*[(15/65-1/3)/(1/3)]^2)',
             (u'6',
              u'4'): u'<pre style="text-align:left; padding-left:.2em">no</pre>',
             (u'6',
              u'5'): u'<pre style=

In [12]:
webwork_parser.parse_webwork('(11!(11-6)')

ERROR:parsetrees.expr_parser.webwork_parser:||(11!(11-6)|| Unbalanced grouping operator in expression: (11!(11-6)
ERROR:parsetrees.expr_parser.webwork_parser:Unbalanced grouping operator in expression: (11!(11-6)
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 710, in parseopt
    p.callable(pslice)
  File "parsetrees/expr_parser/webwork_parser.py", line 228, in p_expression_unbalanced_group
    raise WebworkParseException('Unbalanced grouping operator in expression: ' + t.lexer.lexdata)
WebworkParseException: Unbalanced grouping operator in expression: (11!(11-6)


Parse Error: Unbalanced Group Operator
(11!(11-6)
^


(None, {})

In [21]:
etree = Eval_parsed.eval_parsed(parse_tree[0])

eval_parsed, e=" [['!', [0, 11]], [['*', [1, 9]], [['!', [1, 3]], [['X', [1, 2]], 11]], [['-', [5, 8]], [['X', [5, 6]], 11], [['X', [8, 8]], 6]]]] "
eval_parsed, e=" [['*', [1, 9]], [['!', [1, 3]], [['X', [1, 2]], 11]], [['-', [5, 8]], [['X', [5, 6]], 11], [['X', [8, 8]], 6]]] "
eval_parsed, e=" [['!', [1, 3]], [['X', [1, 2]], 11]] "
eval_parsed, e=" [['X', [1, 2]], 11] "
eval_parsed, e=" 11 "
11.0
11.0
eval_parsed, e=" [['-', [5, 8]], [['X', [5, 6]], 11], [['X', [8, 8]], 6]] "
eval_parsed, e=" [['X', [5, 6]], 11] "
eval_parsed, e=" 11 "
11.0
eval_parsed, e=" [['X', [8, 8]], 6] "
eval_parsed, e=" 6 "
6.0
199584000.0
Eval_parsed Exception: 199584000.0 is too large to apply factorial
Traceback (most recent call last):
  File "parsetrees/expr_parser/Eval_parsed.py", line 97, in eval_parsed
    raise Exception('%s is too large to apply factorial'%v)
Exception: 199584000.0 is too large to apply factorial
